#### Graph

In [1]:
from fugw_solver.solver import solver
import torch
from collections import defaultdict

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

source_adjacency_matrix = np.array([[0, 1, 1, 1, 0, 0, 0, 0],
                             [1, 0, 0, 1, 0, 0, 1, 0],
                             [1, 0, 0, 1, 1, 0, 0, 0],
                             [1, 1, 1, 0, 0, 0, 1, 0],
                             [0, 0, 1, 0, 0, 1, 0, 1],
                             [0, 0, 0, 0, 1, 0, 1, 0],
                             [0, 1, 0, 1, 0, 1, 0, 0],
                             [0, 0, 0, 0, 1, 0, 0, 0]])

n_points_source = len(source_adjacency_matrix)

source_graph = nx.Graph(source_adjacency_matrix)

source_locations = nx.spring_layout(source_graph)

source_features = np.ones(n_points_source)

plt.figure(figsize=(3, 3))
nx.draw(source_graph, source_locations, with_labels=True)
plt.title('Complex Graph 1 with Node Positions')
plt.show()

In [ ]:
target_adjacency_matrix = np.array([
                             [0, 0, 1, 0, 0, 1, 0],
                             [0, 0, 1, 1, 0, 0, 0],
                             [1, 1, 0, 0, 1, 1, 0],
                             [0, 1, 0, 0, 1, 0, 1],
                             [0, 0, 1, 1, 0, 1, 0],
                             [1, 0, 1, 0, 1, 0, 0],
                             [0, 0, 0, 1, 0, 0, 0]])

n_points_target = len(target_adjacency_matrix)

target_graph = nx.Graph(target_adjacency_matrix)

target_locations = nx.spring_layout(target_graph)

target_features = np.ones(n_points_target)

plt.figure(figsize=(3, 3))
nx.draw(target_graph, target_locations, with_labels=True)
plt.title('Complex Graph 1 with Node Positions')
plt.show()

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def plot_graph_matching_3D(source_graph, target_graph, P, distance_between_graphs):
    pos_source = nx.spring_layout(source_graph)  
    pos_target = nx.spring_layout(target_graph)  

    pos_source = np.array([[pos_source[key][0], pos_source[key][1], 0] for key in pos_source.keys()])
    pos_target = np.array([[pos_target[key][0], pos_target[key][1], distance_between_graphs] for key in pos_target.keys()])
    
    edges_source = list(source_graph.edges())
    segments_source = [[pos_source[edge[0]].tolist(), pos_source[edge[1]].tolist()] for edge in edges_source]
    
    edges_target = list(target_graph.edges())
    segments_target = [[pos_target[edge[0]].tolist(), pos_target[edge[1]].tolist()] for edge in edges_target]
    
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111, projection='3d')  
    
    line_segments_source = Line3DCollection(segments_source, colors="black", lw=1, zorder=1)
    ax.add_collection(line_segments_source)  
    ax.scatter(pos_source[:, 0], pos_source[:, 1], pos_source[:, 2], color='blue', s=200, label='Source Nodes')
    
    line_segments_target = Line3DCollection(segments_target, colors="black", lw=1, zorder=1)
    ax.add_collection(line_segments_target)  
    ax.scatter(pos_target[:, 0], pos_target[:, 1], pos_target[:, 2], color='orange', s=200, label='Target Nodes')

    transport_matrix = P / P.sum(dim=1).reshape(-1, 1)

    for i in range(transport_matrix.shape[0]):
        for j in range(transport_matrix.shape[1]):
            x1, y1, z1 = pos_source[i]
            x2, y2, z2 = pos_target[j]
            ax.plot([x1, x2], [y1, y2], [z1, z2], color='red', alpha=float(transport_matrix[i, j]))

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')
    ax.set_title('Graph Matching')
    ax.legend()
    plt.show()

In [ ]:
%matplotlib inline
plot_graph_matching_3D(source_graph, target_graph, P, 3)

#### Sphere

In [1]:
# Checkpoint
from fugw_solver.solver import solver
import torch
from collections import defaultdict

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [2]:
# Functions

def mises_density(A, mu, kappa=1.0):
    cste = kappa / (4 * np.pi * np.sinh(kappa))
    density = cste * np.exp(kappa * np.dot(A,mu.T))
    density = density / np.sum(density)     # Normalize the density
    return density

def create_sphere(cx, cy, cz, r, resolution=360):
    phi = np.linspace(0, 2*np.pi, 2*resolution)
    theta = np.linspace(0, np.pi, resolution)
    theta, phi = np.meshgrid(theta, phi)
    r_xy = r * np.sin(theta)
    x = cx + np.cos(phi) * r_xy
    y = cy + np.sin(phi) * r_xy
    z = cz + r * np.cos(theta)
    return x, y, z

def fibonacci_sphere(samples=1000, randomize=True):
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

    if randomize:
        theta = np.random.rand(samples) * 2 * np.pi  # random angles
        z = np.random.rand(samples) * 2 - 1  # random z values
    else:
        indices = np.arange(0, samples, dtype=float) + 0.5
        theta = phi * indices
        z = np.linspace(1 - 1.0 / samples, 1.0 / samples - 1, samples)

    radius = np.sqrt(1 - z * z)

    points = np.zeros((samples, 3))
    points[:, 0] = radius * np.cos(theta)
    points[:, 1] = radius * np.sin(theta)
    points[:, 2] = z

    # return points

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    return x, y, z

In [3]:
cx, cy, cz = 0, 0, 0  # Center of the sphere
radius = 1.0
resolution = 30

# x, y, z = create_sphere(cx, cy, cz, radius, resolution)
x, y, z = fibonacci_sphere(samples=1000, randomize=False)

kappa_1 = 5.0  
mu_1 = np.array([0., -1., 0.])
A = np.column_stack((x.flatten(), y.flatten(), z.flatten()))
scalar_values_1 = mises_density(A, mu_1, kappa_1)


kappa_2 = 10.0  
mu_2 = np.array([0., 0., 1.])
scalar_values_2 = mises_density(A, mu_2, kappa_2)

scalar_values = 0.6 * scalar_values_1 + 0.4 * scalar_values_2

print(np.sum(scalar_values))

# Plot the sphere with a color gradient based on scalar values
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x, y, z, c=scalar_values, cmap='viridis', s=20)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_aspect('equal')
ax.set_title('Sphere Plot with Mises Density')

# Add a colorbar to show the density variation
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Density')

plt.show()

: 

In [ ]:
scalar_values = scalar_values / np.linalg.norm(scalar_values)
scalar_values_1 = scalar_values_1 / np.linalg.norm(scalar_values_1)

source_features_normalized = torch.tensor(scalar_values, dtype=torch.float32).unsqueeze(0)
target_features_normalized = torch.tensor(scalar_values_1, dtype=torch.float32).unsqueeze(0)

print(source_features_normalized.sum(), target_features_normalized.sum())

source_geometry = torch.tensor(A, dtype=torch.float32)
source_geometry = torch.cdist(source_geometry, source_geometry)
target_geometry = source_geometry

w_s = torch.ones(source_features_normalized.shape[1], device='cpu') / source_features_normalized.shape[1]
w_t = torch.ones(target_features_normalized.shape[1], device='cpu') / target_features_normalized.shape[1]

In [ ]:
alpha = 0.5
rho = 1
eps = 1e-4
mapping = solver(alpha=alpha, rho=rho, eps=eps, 
    source_features=source_features_normalized,
    target_features=target_features_normalized,
    source_geometry=source_geometry,
    target_geometry=target_geometry,
    device='cpu',
    w_s=w_s,
    w_t=w_t
)

In [ ]:
P, loss_ls = mapping

loss = defaultdict(list)
for l in loss_ls:
    for k,v in l.items():
        loss[k].append(v)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.set_title("Mapping training loss")
ax.set_ylabel("Loss")
ax.set_xlabel("BCD step")
ax.stackplot(
    np.arange(len(loss_ls)),
    [
        (1 - alpha) * np.array(loss["wasserstein"]),
        alpha * np.array(loss["gromov_wasserstein"]),
        rho * np.array(loss["marginal_constraint_dim1"]),
        rho * np.array(loss["marginal_constraint_dim2"]),
        eps * np.array(loss["regularization"]),
    ],
    labels=[
        "wasserstein",
        "gromov_wasserstein",
        "marginal_constraint_dim1",
        "marginal_constraint_dim2",
        "regularization",
    ],
    alpha=0.8,
)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_title("Transport plan")
ax.set_xlabel("target vertices")
ax.set_ylabel("source vertices")
im = plt.imshow(P, cmap="viridis")
plt.colorbar(im, ax=ax, shrink=0.8)
plt.show()

In [ ]:
P.sum()

In [ ]:
predicted_features = P.T@source_features_normalized.T
predicted_features = predicted_features.squeeze(0)

In [ ]:
predicted_features.sum()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x, y, z, c=predicted_features, cmap='viridis', s=20)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_aspect('equal')
ax.set_title('Sphere Plot with Mises Density')

# Add a colorbar to show the density variation
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Density')

plt.show()